# Tarefa 02 Módulo 05

O nosso projeto desta sequência de módulos do curso será um aprofundamento da demonstração sobre classificação de risco de crédito que vimos lá no comecinho. Pois recebemos uma base já montada pra nós. Tenha certeza de que ela passou por um longo processamento até ficar daquele jeito. Neste exercício vamos exercitar o que aprendemos nas ultimas aulas e montar a variável resposta da base do nosso projeto.

#### Marcação de bom e mau
O objetivo da modelagem é classificar o risco de inadimplência, ou como se diz no meio, o risco de *default*. Podemos fazer longas discussões sobre o conceito de *default* com base em estudos e exigências regulatórias, para efeitos deste estudo, um cliente em *default* é aquele que está em 60 dias de atraso ou mais. Então classificaremos os clientes como 'bons' e 'maus' assim:
- **Maus** pagadores: são aqueles que entraram em 'default' (atraso 60 dias ou mais) nos 24 meses seguintes à aquisição do cartão de crédito.
- **Bons** pagadores: são considerados todos os demais.
- **Excluídos**: Clientes que não adquiriram um cartão de crédito (seja por recusa, seja por desistência) não possuem informações de pagamento, portanto não se pode identificar se são bons ou maus. Há uma longa discussão e literatura sobre *inferência de rejeitados* que está fora do escopo deste exercício.

#### Bases disponíveis
Temos duas bases importantes aqui: uma de propostas, com diversas informações dos vários solicitantes de cartão de crédito, e uma base de pagamentos. A base de pagamentos será utilizada para identificar a ocorrência de *default*. A base de propostas tem diversas informações coletadas no momento da solicitação do crédito (isto é importante: qualquer informação posterior a essa data é impossível de ser coletada na aplicação do modelo e não pode ser utilizada).

As variáveis delas são:

Base de propostas - application_records.csv

| Nome da Variável         | Description                                         | Tipo  |
| ------------------------ |:---------------------------------------------------:| -----:|
| ID| identificador do cliente (chave) |inteiro|
| CODE_GENDER| M = 'Masculino'; F = 'Feminino' |M/F|
| FLAG_OWN_CAR| Y = 'possui'; N = 'não possui' |Y/N|
| FLAG_OWN_REALTY| Y = 'possui'; N = 'não possui' |Y/N|
| CNT_CHILDREN| Quantidade de filhos |inteiro|
| AMT_INCOME_TOTAL| Annual income |inteiro|
| NAME_INCOME_TYPE|Tipo de renda (ex: assaliariado, autônomo etc) | texto |
| NAME_EDUCATION_TYPE| Nível de educação (ex: secundário, superior etc) |texto|
| NAME_FAMILY_STATUS | Estado civil (ex: solteiro, casado etc)| texto |
| NAME_HOUSING_TYPE | tipo de residência (ex: casa/apartamento, com os pais etc) | texto |
| DAYS_BIRTH | Count backwards from current day (0), -1 means yesterday |inteiro|
| DAYS_EMPLOYED | Count backwards from current day (0), -1 means yesterday |inteiro|
| FLAG_MOBIL | Indica se possui celular (1 = sim, 0 = não) |binária|
| FLAG_WORK_PHONE | Indica se possui telefone comercial (1 = sim, 0 = não) |binária|
| FLAG_PHONE | Indica se possui telefone (1 = sim, 0 = não) |binária|
| FLAG_EMAIL | Indica se possui e-mail (1 = sim, 0 = não) |binária|
| OCCUPATION_TYPE | Occupation	 |Qualitativa|
| CNT_FAM_MEMBERS | quantidade de pessoas na residência |inteiro|

Base de pagamentos - pagamentos_largo.csv  

| Nome da Variável         | Description                                         | Tipo  |
| ------------------------ |:---------------------------------------------------:| -----:|
| ID| identificador do cliente (chave) |inteiro|
| mes_00 a mes_24| faixa de atraso mês a mês do cliente <br>0: 1-29 days past due &nbsp;&nbsp;&nbsp;&nbsp; 1: 30-59 days past due <br />2: 60-89 days overdue &nbsp;&nbsp;&nbsp;&nbsp; 3: 90-119 days overdue <br /> 4: 120-149 days overdue &nbsp;&nbsp;&nbsp;&nbsp; 5: more than 150 days <br />C: paid off that month &nbsp;&nbsp;&nbsp;&nbsp; X: No loan for the month |Qualitativa|

#### Construindo a variável resposta
A base de pagamentos está em um formato de 'base larga'. Essa base possui informações de pagamentos do cliente mês a mês a partir do mês de aquisição do crédito (mês 0) até o vigésimo quarto mês após a aquisição do crédito (mês 24). Utilizaremos essa base para determinar se um proponente é considerado 'bom pagador' ou caso apresente atraso representativo, será considerado 'mau pagador'.

#### Base larga vs base longa
A base ser larga significa que há uma linha para cada cliente, e que as informações estarão nas colunas, em contraste com a 'base longa', em que haveria uma linha para cada combinação cliente/mês, uma coluna indicando o cliente, outra indicando o mês, e apenas uma coluna com a informação do atraso.

In [1]:
import pandas as pd

In [2]:
# Importando a base de dados de pagamento
link = "https://raw.githubusercontent.com/gfsilveira/ebac/main/"
link += "03-Cientista%20de%20Dados/02-Crisp-DM/M%C3%B3dulo_06/"
link += "Tarefa%202/pagamentos_largo.csv"
pg = pd.read_csv(link)
pg.head()

,ID,mes_0,mes_1,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,...,mes_22,mes_23,mes_24,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,mes_9
0,5001718,0,0,0,0,0,0,0,0,NaN,...,NaN,0,NaN,0,0,0,0,NaN,0,NaN
1,5001719,0,0,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,C,C
2,5001720,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,5001723,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,NaN,NaN
4,5001726,0,0,C,C,C,C,C,C,C,...,C,C,C,0,0,0,C,C,C,C


In [3]:
# Verificando a base
pg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20937 entries, 0 to 20936
Data columns (total 26 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      20937 non-null  int64 
 1   mes_0   20937 non-null  object
 2   mes_1   19216 non-null  object
 3   mes_10  17455 non-null  object
 4   mes_11  16972 non-null  object
 5   mes_12  16943 non-null  object
 6   mes_13  16540 non-null  object
 7   mes_14  16525 non-null  object
 8   mes_15  16155 non-null  object
 9   mes_16  16093 non-null  object
 10  mes_17  15911 non-null  object
 11  mes_18  15630 non-null  object
 12  mes_19  15336 non-null  object
 13  mes_2   19348 non-null  object
 14  mes_20  15184 non-null  object
 15  mes_21  15076 non-null  object
 16  mes_22  14797 non-null  object
 17  mes_23  14598 non-null  object
 18  mes_24  14433 non-null  object
 19  mes_3   18925 non-null  object
 20  mes_4   18802 non-null  object
 21  mes_5   18512 non-null  object
 22  mes_6   18214 non-null



---



## 1 Marcar *default* no mês
Faça uma indicadora de se o cliente está em *default* em cada uma das marcações (mes_00 a mes_24). Dica: você pode utilizar o método ```.isin()``` do Pandas. Consulte a [documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html) caso necessário.

In [4]:
# 1) Seu código aqui
'''
Busca, com a função isin(), na DataFrame pg, os clientes com código 2 a 5,
como referência de pagamentos em atrasa com mais de 60 dias.
'''
default = ["2","3","4","5"]
pg_default = pg.copy()
pg_default[pg_default.columns[1:]] = pg_default.iloc[:,1:].isin(default)
pg_default.head()

,ID,mes_0,mes_1,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,...,mes_22,mes_23,mes_24,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,mes_9
0,5001718,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,5001719,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,5001720,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,5001723,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,5001726,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False




---



## 2 'Bons' e 'maus' ao longo de todos os 24 meses de desempenho
Marque para cada cliente se ele teve pelo menos um episódio de *default* entre o mês 0 e o mês 24. Dica: o método ```sum()``` pode ajudar. Caso precise, consulte a [documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sum.html) e procure pelo argumento ```axis```, você viu outros métodos que possuem esse argumento também. Tendo o número de meses em default de cada cliente, basta marcar ```True``` para todos aqueles que possuem pelo menos 1 mês em *default* e ```False``` para os demais.

In [5]:
# 2) Seu código aqui
'''
Somando o número de True na DataFrame, sem o número de ID dos clientes,
por columa com sum(axis='columns').
'''
pg_default['default'] = pg_default.iloc[:,1:].sum(axis='columns')

'''
Criando a variável default com apply, substuindo 0 por False e qualquer outro
valor por True, assim, caso o cliente tenha entrado em defalt qualquer um dos
meses, será registrado como True.
'''
pg_default['default'] = pg_default['default'].apply(lambda i: False if i == 0 else True)

# Contando o número de entradas
pg_default['default'].value_counts()

False    20506
True       431
Name: default, dtype: int64

In [6]:
# Repetindo a análise acima, agora com isin e any por linhas
pg_default['default_2'] = pg_default.T.isin([True]).any()

# Contando o número de entradas
pg_default['default_2'].value_counts()

False    20506
True       431
Name: default_2, dtype: int64

In [7]:
'''
A comparação das técnicas mostra que elas são equivalentes aos dados originais,
isto é, não se perdeu informação no processamento.
'''
len(pg_default[pg_default['default'] == pg_default['default']]) == len(pg)

True



---



## 3 Marcando proponentes expostos ao risco de crédito
Marcando proponentes que se tornaram tomadores: lembre-se de que clientes que não adquiriram o cartão devem ser desconsiderados. A base de pagamentos possui apenas clientes que adquiriram cartão de crédito, então você pode selecionar somente os clientes da base de propostas que se encontram na base de pagamentos.

In [8]:
# 3) Seu código aqui
# Verificando as variáveis de interesse
pg_default[['ID','default']].head()

,ID,default
0,5001718,False
1,5001719,False
2,5001720,False
3,5001723,False
4,5001726,False


In [9]:
# Importando os dados
link = "https://raw.githubusercontent.com/gfsilveira/ebac/main/"
link += "03-Cientista%20de%20Dados/02-Crisp-DM/M%C3%B3dulo_06/"
link += "Tarefa%202/application_record.csv"

propostas = pd.read_csv(link)
propostas.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [10]:
# Copiando a DataFrame original, inserindo os dados de default, verificando.
propostas_default = propostas.copy()
propostas_default = propostas_default.merge(
                                                pg_default[['ID','default']],
                                                on='ID',
                                                how='left'
                                          )
propostas_default.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,default
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,NaN
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,NaN
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,False
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,NaN
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,NaN


In [11]:
# Aparentemente não foram perdidas informações
propostas_default['default'].value_counts(dropna=False).sum() == len(propostas)

True



---



## 4 Consolidando as informações
Faça uma junção das informações da base de propostas com a variável de *default* que você acabou de construir. Talvez você consiga realizar a tarefa 3 e tarefa 4 em uma única linha de código ;)

In [12]:
# 4) eu código aqui
# Feito acima, top, :)



---



## 5 Verificando
Faça uma contagem dos valores do *default* que você construiu.

In [13]:
'''
Contando o número de clientes em defaults. Dos que adiquiriram, cerca de 2,3%
entraram em default
'''
propostas_default['default'].value_counts(normalize=True)*100

False    97.657658
True      2.342342
Name: default, dtype: float64

In [14]:
# Contando o número de clientes em defaults.
propostas_default['default'].value_counts(dropna=False)

NaN      421907
False     16260
True        390
Name: default, dtype: int64

In [15]:
'''
Aparentemente, a grande maioria dos clientes (~96%) não adquiriram o cartão.
'''
propostas_default['default'].value_counts(normalize=True, dropna=False)*100

NaN      96.203458
False     3.707614
True      0.088928
Name: default, dtype: float64